In [1]:
import pandas
import numpy
import requests
import json

In [2]:
url = "https://platform-api.opentargets.io/v3/platform/private/target/ENSG00000087085"
raw_api_response = requests.get(url).json()
pretty_api_response = json.dumps(raw_api_response, indent=2, sort_keys=True)
print(pretty_api_response)

{
  "alias_name": [],
  "alias_symbol": [],
  "approved_name": "acetylcholinesterase (Cartwright blood group)",
  "approved_symbol": "ACHE",
  "biotype": "protein_coding",
  "ccds_ids": [],
  "chembl": [],
  "chemicalprobes": {
    "probeminer": {
      "link": "https://probeminer.icr.ac.uk/#/P22303",
      "probenumber": "3275"
    }
  },
  "chromosome": "7",
  "cytobands": "",
  "dbxrefs": [
    "BRENDA:3.1.1.7",
    "Bgee:ENSG00000087085",
    "BindingDB:P22303",
    "BioGrid:106561",
    "BioMuta:ACHE",
    "CCDS:CCDS5709.1",
    "CCDS:CCDS5710.1",
    "CCDS:CCDS64736.1",
    "CTD:43",
    "ChEMBL:CHEMBL220",
    "ChiTaRS:ACHE",
    "DIP:DIP-1119N",
    "DMDM:113037",
    "DOI:10.1006/excr.1994.1039",
    "DOI:10.1016/0014-5793(89)81352-3",
    "DOI:10.1016/s0168-0102(02)00005-6",
    "DOI:10.1016/s1093-3263(98)00005-9",
    "DOI:10.1021/jm300871x",
    "DOI:10.1021/jm901853b",
    "DOI:10.1038/nature01782",
    "DOI:10.1038/ng1285",
    "DOI:10.1038/sj.emboj.7600425",
    "DOI:10.

In [3]:
print(raw_api_response["biotype"])

protein_coding


In [4]:
important_fields = [
    "approved_name",
    "biotype",
    "uniprot_id"
]

all_data = []

data_row = []

for field in important_fields:
    data_row.append(raw_api_response[field])

all_data.append(data_row)

print(all_data)

[['acetylcholinesterase (Cartwright blood group)', 'protein_coding', 'P22303']]


In [5]:
df = pandas.DataFrame(all_data)
df.columns = important_fields
df.head()

,approved_name,biotype,uniprot_id
0,acetylcholinesterase (Cartwright blood group),protein_coding,P22303


In [6]:
input_data = [
    {
        "target_symbol": "PIK3CA",
        "target_id": "ENSG00000121879",
        "diseases": [
            {
                "efo_label": "breast carcinoma",
                "efo_id": "EFO_0000305"
            },
            {
                "efo_label": "colorectal cancer",
                "efo_id": "EFO_1001951",
            },
        ]
    },
    {
        "target_symbol": "AKT1",
        "target_id": "ENSG00000142208",
        "diseases": [
            {
                "efo_label": "breast carcinoma",
                "efo_id": "EFO_0000305"
            }
        ]
    },
    {
        "target_symbol": "APC",
        "target_id": "ENSG00000134982",
        "diseases": [
            {
                "efo_label": "colorectal cancer",
                "efo_id": "EFO_1001951",
            }
        ]
    }
]

In [11]:
important_fields = [
    "id",
    "approved_symbol",
    "approved_name",
    "biotype",
    "uniprot_id"
]

all_data = []

base_url_platform_gene_index = "https://platform-api.opentargets.io/v3/platform/private/target/"

for entry in input_data:
    # generate URL for API request
    gene_id = entry["target_id"]
    url = base_url_platform_gene_index + gene_id
    print("API request URL: " + url)
    # make API request
    raw_api_response = requests.get(url).json()
    # create row of data
    data_row = []
    for field in important_fields:
        data_row.append(raw_api_response[field])
    all_data.append(data_row)

df = pandas.DataFrame(all_data)
df.columns = important_fields
df.head()

API request URL: https://platform-api.opentargets.io/v3/platform/private/target/ENSG00000121879
API request URL: https://platform-api.opentargets.io/v3/platform/private/target/ENSG00000142208
API request URL: https://platform-api.opentargets.io/v3/platform/private/target/ENSG00000134982


,id,approved_symbol,approved_name,biotype,uniprot_id
0,ENSG00000121879,PIK3CA,"phosphatidylinositol-4,5-bisphosphate 3-kinase...",protein_coding,P42336
1,ENSG00000142208,AKT1,AKT serine/threonine kinase 1,protein_coding,P31749
2,ENSG00000134982,APC,APC regulator of WNT signaling pathway,protein_coding,P25054


In [26]:
all_data2 = []

base_url_platform_drug_endpoint = "https://platform-api.opentargets.io/v3/platform/public/evidence/filter?size=10000&datasource=chembl&fields=evidence.drug2clinic.clinical_trial_phase&fields=evidence.drug2clinic.urls&fields=drug.id&target="

for entry in input_data:
    # generate URL for API request
    gene_id = entry["target_id"]
    url = base_url_platform_drug_endpoint + gene_id
    print("API request URL: " + url)
    data_row = []
    data_row.append(gene_id)
    raw_api_response = requests.get(url).json()
    if raw_api_response["data"]:
        data_row.append(True)
        drugs = []
        for obj in raw_api_response["data"]:
            drug_url = obj["drug"]["id"]
            drug_url_strings = drug_url.split("/")
            chembl_id = drug_url_strings[-1]
            drugs.append(chembl_id)
        drugs_list = list(set(drugs))
        all_drugs = ",".join(drugs_list)
        data_row.append(all_drugs)
    else:
        data_row.append(False)
    all_data2.append(data_row)

API request URL: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?size=10000&datasource=chembl&fields=evidence.drug2clinic.clinical_trial_phase&fields=evidence.drug2clinic.urls&fields=drug.id&target=ENSG00000121879
API request URL: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?size=10000&datasource=chembl&fields=evidence.drug2clinic.clinical_trial_phase&fields=evidence.drug2clinic.urls&fields=drug.id&target=ENSG00000142208
API request URL: https://platform-api.opentargets.io/v3/platform/public/evidence/filter?size=10000&datasource=chembl&fields=evidence.drug2clinic.clinical_trial_phase&fields=evidence.drug2clinic.urls&fields=drug.id&target=ENSG00000134982


In [27]:
df2 = pandas.DataFrame(all_data2)
df2.head()

,0,1,2
0,ENSG00000121879,True,"CHEMBL411907,CHEMBL1879463,CHEMBL3545248,CHEMB..."
1,ENSG00000142208,True,"CHEMBL3545134,CHEMBL2178577,CHEMBL494089,CHEMB..."
2,ENSG00000134982,False,None


In [28]:
df2.columns = ["id", "known_drug_in_clinic", "drugs_in_clinic"]

In [29]:
df2.head()

,id,known_drug_in_clinic,drugs_in_clinic
0,ENSG00000121879,True,"CHEMBL411907,CHEMBL1879463,CHEMBL3545248,CHEMB..."
1,ENSG00000142208,True,"CHEMBL3545134,CHEMBL2178577,CHEMBL494089,CHEMB..."
2,ENSG00000134982,False,None


In [30]:
df3 = pandas.merge(df, df2, on="id")
df3.head()

,id,approved_symbol,approved_name,biotype,uniprot_id,known_drug_in_clinic,drugs_in_clinic
0,ENSG00000121879,PIK3CA,"phosphatidylinositol-4,5-bisphosphate 3-kinase...",protein_coding,P42336,True,"CHEMBL411907,CHEMBL1879463,CHEMBL3545248,CHEMB..."
1,ENSG00000142208,AKT1,AKT serine/threonine kinase 1,protein_coding,P31749,True,"CHEMBL3545134,CHEMBL2178577,CHEMBL494089,CHEMB..."
2,ENSG00000134982,APC,APC regulator of WNT signaling pathway,protein_coding,P25054,False,None


In [31]:
df3["drugs_in_clinic"].value_counts()

CHEMBL3545134,CHEMBL2178577,CHEMBL494089,CHEMBL2177390,CHEMBL462018,CHEMBL3544935,CHEMBL3545049,CHEMBL3545422,CHEMBL3545003,CHEMBL2219422,CHEMBL1079175,CHEMBL3137336                                                                                                                                                                                                                                  1
CHEMBL411907,CHEMBL1879463,CHEMBL3545248,CHEMBL2326966,CHEMBL3544966,CHEMBL3360203,CHEMBL1922094,CHEMBL3218578,CHEMBL1258517,CHEMBL3544999,CHEMBL3545322,CHEMBL3545245,CHEMBL1684984,CHEMBL2396661,CHEMBL2387080,CHEMBL3622533,CHEMBL592445,CHEMBL1236962,CHEMBL586701,CHEMBL1079593,CHEMBL3545324,CHEMBL3218576,CHEMBL521851,CHEMBL3545379,CHEMBL3545366,CHEMBL3545385,CHEMBL2017974,CHEMBL1234354    1
Name: drugs_in_clinic, dtype: int64

In [32]:
my_val = df3.iloc[0]['drugs_in_clinic']
print(my_val)

CHEMBL411907,CHEMBL1879463,CHEMBL3545248,CHEMBL2326966,CHEMBL3544966,CHEMBL3360203,CHEMBL1922094,CHEMBL3218578,CHEMBL1258517,CHEMBL3544999,CHEMBL3545322,CHEMBL3545245,CHEMBL1684984,CHEMBL2396661,CHEMBL2387080,CHEMBL3622533,CHEMBL592445,CHEMBL1236962,CHEMBL586701,CHEMBL1079593,CHEMBL3545324,CHEMBL3218576,CHEMBL521851,CHEMBL3545379,CHEMBL3545366,CHEMBL3545385,CHEMBL2017974,CHEMBL1234354
